In [1]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import os
import zipfile
from urllib.parse import urljoin
import xlsxwriter
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def get_subpages(base_url):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, base_url)
        soup = BeautifulSoup(html, 'html.parser')
        subpages = set()  # Use a set to avoid duplicates
        
        # Find all links on the page in the order they appear
        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.startswith('/pages/forms/') and href != '/pages/forms/':
                full_url = urljoin(base_url, href)
                subpages.add(full_url)  # Add to set to ensure uniqueness
        return list(subpages)

async def save_subpages(subpages):
    if not os.path.exists('HockeyTeams'):
        os.makedirs('HockeyTeams')
    
    all_data = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i, url in enumerate(subpages, 1):
            tasks.append(fetch(session, url))
        responses = await asyncio.gather(*tasks)
        
        for i, html in enumerate(responses, 1):
            filename = f'HockeyTeams/{i}.html'
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(html)

            # Extract structured data from the HTML table rows
            soup = BeautifulSoup(html, 'html.parser')
            table = soup.find('table')
            if table:
                headers = [th.text.strip() for th in table.find_all('th')]
                for row in table.find_all('tr')[1:]:  # Skip header row
                    columns = row.find_all('td')
                    if columns:
                        row_data = {headers[i]: columns[i].text.strip() for i in range(len(columns))}
                        all_data.append(row_data)
    
    # Remove duplicates from all_data
    unique_data = [dict(t) for t in {tuple(sorted(row.items())) for row in all_data}]
    return headers, unique_data

def create_zip():
    with zipfile.ZipFile('HockeyTeams.zip', 'w') as zipf:
        for root, dirs, files in os.walk('HockeyTeams'):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), 'HockeyTeams'))

def generate_winner_loser_summary(all_data):
    summary = {}
    for row in all_data:
        year = row.get('Year')
        team = row.get('Team Name')
        wins = row.get('Wins', '0').strip()
        if year and team and wins.isdigit():
            wins = int(wins)
            if year not in summary:
                summary[year] = {}
            summary[year][team] = wins
    
    winner_loser_data = []
    for year, teams in sorted(summary.items()):  # Sort by year
        if teams:  # Ensure there are valid teams
            winner = max(teams.items(), key=lambda x: x[1])  # Team with most wins
            loser = min(teams.items(), key=lambda x: x[1])   # Team with least wins
            winner_loser_data.append([
                year, winner[0], winner[1], loser[0], loser[1]
            ])
        else:
            logging.warning(f"No valid data for year {year}")
    return winner_loser_data

def save_to_excel(headers, all_data):
    workbook = xlsxwriter.Workbook('HockeyTeams_data.xlsx')
    
    # Sheet 1: NHL Stats 1990-2011
    worksheet1 = workbook.add_worksheet('NHL Stats 1990-2011')
    for col_num, header in enumerate(headers):
        worksheet1.write(0, col_num, header)
    for row_num, row_data in enumerate(all_data, start=1):
        for col_num, header in enumerate(headers):
            worksheet1.write(row_num, col_num, row_data.get(header, ''))
    
    # Sheet 2: Winner and Loser per Year
    worksheet2 = workbook.add_worksheet('Winner and Loser per Year')
    summary_headers = ['Year', 'Winner', 'Winner Num. of Wins', 'Loser', 'Loser Num. of Wins']
    for col_num, header in enumerate(summary_headers):
        worksheet2.write(0, col_num, header)
    
    summary_data = generate_winner_loser_summary(all_data)
    for row_num, summary_row in enumerate(summary_data, start=1):
        for col_num, cell_data in enumerate(summary_row):
            worksheet2.write(row_num, col_num, cell_data)
    
    workbook.close()

async def main():
    base_url = 'https://www.scrapethissite.com/pages/forms/'
    subpages = await get_subpages(base_url)
    headers, all_data = await save_subpages(subpages)
    create_zip()
    save_to_excel(headers, all_data)
    logging.info("Subpages have been saved, zipped, and actual data extracted to Excel successfully with two sheets: 'NHL Stats 1990-2011' and 'Winner and Loser per Year'.")

if __name__ == '__main__':
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

2025-02-23 11:32:58,106 - INFO - Subpages have been saved, zipped, and actual data extracted to Excel successfully with two sheets: 'NHL Stats 1990-2011' and 'Winner and Loser per Year'.
